In [3]:
import sys
import sqlite3
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QTableView, QVBoxLayout, QPushButton,
    QLineEdit, QWidget, QHBoxLayout, QFormLayout, QDialog, QLabel, QMessageBox
)
from PyQt5.QtCore import QAbstractTableModel, Qt

In [4]:
class PostsTableModel(QAbstractTableModel):
    def __init__(self, data):
        super().__init__()
        self._data = data

    def data(self, index, role):
        if role == Qt.DisplayRole:
            return self._data[index.row()][index.column()]

    def rowCount(self, index):
        return len(self._data)

    def columnCount(self, index):
        return len(self._data[0])

    def headerData(self, section, orientation, role):
        headers = ["ID", "User ID", "Title", "Body"]
        if role == Qt.DisplayRole:
            if orientation == Qt.Horizontal:
                return headers[section]

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.conn = sqlite3.connect('labDataBase.db')
        self.cursor = self.conn.cursor()

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        self.table_view = QTableView()
        self.search_bar = QLineEdit()
        self.search_bar.setPlaceholderText("Поиск по заголовку...")
        self.refresh_button = QPushButton("Обновить")
        self.add_button = QPushButton("Добавить")
        self.delete_button = QPushButton("Удалить")

        layout = QVBoxLayout()
        button_layout = QHBoxLayout()

        button_layout.addWidget(self.refresh_button)
        button_layout.addWidget(self.add_button)
        button_layout.addWidget(self.delete_button)

        layout.addWidget(self.search_bar)
        layout.addWidget(self.table_view)
        layout.addLayout(button_layout)

        self.central_widget.setLayout(layout)

        self.refresh_button.clicked.connect(self.refresh_data)
        self.add_button.clicked.connect(self.show_add_dialog)
        self.delete_button.clicked.connect(self.delete_record)
        self.search_bar.textChanged.connect(self.search_title)

        self.refresh_data()

    def refresh_data(self):
        query = "SELECT * FROM posts"
        self.cursor.execute(query)
        records = self.cursor.fetchall()
        self.update_table(records)

    def update_table(self, records):
        self.model = PostsTableModel(records)
        self.table_view.setModel(self.model)

    def search_title(self):
        search_text = self.search_bar.text()
        query = "SELECT * FROM posts WHERE title LIKE ?"
        self.cursor.execute(query, ('%' + search_text + '%',))
        records = self.cursor.fetchall()
        self.update_table(records)

    def show_add_dialog(self):
        dialog = AddPostDialog(self)
        dialog.exec_()

    def add_post(self, user_id, title, body):
        query = "INSERT INTO posts (user_id, title, body) VALUES (?, ?, ?)"
        self.cursor.execute(query, (user_id, title, body))
        self.conn.commit()
        self.refresh_data()

    def delete_record(self):
        selected = self.table_view.selectionModel().selectedRows()
        if selected:
            reply = QMessageBox.question(self, "Подтверждение", "Удалить выбранную запись?",
                                         QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
            if reply == QMessageBox.Yes:
                for index in selected:
                    post_id = self.model.data(index, Qt.DisplayRole)
                    query = "DELETE FROM posts WHERE id = ?"
                    self.cursor.execute(query, (post_id,))
                self.conn.commit()
                self.refresh_data()

class AddPostDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Добавить запись")

        self.user_id_input = QLineEdit()
        self.title_input = QLineEdit()
        self.body_input = QLineEdit()

        self.add_button = QPushButton("Добавить")
        self.add_button.clicked.connect(self.add_post)

        layout = QFormLayout()
        layout.addRow("User ID:", self.user_id_input)
        layout.addRow("Title:", self.title_input)
        layout.addRow("Body:", self.body_input)
        layout.addRow(self.add_button)

        self.setLayout(layout)

    def add_post(self):
        user_id = self.user_id_input.text()
        title = self.title_input.text()
        body = self.body_input.text()

        if user_id and title and body:
            self.parent().add_post(user_id, title, body)
            self.accept()
        else:
            QMessageBox.warning(self, "Ошибка", "Заполните все поля")

In [ ]:
if __name__ == "__main__":
    app = QApplication(sys.argv)

    window = MainWindow()
    window.setWindowTitle("Lab 4")
    window.resize(800, 600)
    window.show()

    sys.exit(app.exec_())